In [1]:
import os
import pandas as pd
import os
import pandas as pd
import numpy as np
from scipy import stats
import openpyxl
import matplotlib.pyplot as plt
import scipy
from scipy.fftpack import irfft, rfft

%matplotlib inline
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import seaborn as sns

In [2]:
#### load anand old file (by arun)

old = pd.read_excel('./Induced Failures in the test Loop- includes all  Updates(Updated Arun).xlsx', sheet_name='Induced Failures',header=0)
old.head(3)

,Entry Date,Defects Introduced,Conveyor Type,Asset-Id,Sensor Location,Macid,Start Time,End Time,Time Difference,Remarks,...,Roller_Wheels-Inner Trunk Roller,Roller_Wheels-Upper Trunk Roller,Roller_Wheels-RW Wheels,TensionUnit-RW-Bearing,TensionUnit-Tension Spring,TensionUnit-Tension Shaft / Unit,SlaveShaft-Bearing,SlaveShaft-Wheels,SlaveShaft- Shaft,SlaveShaft-Remarks2
0,2020-07-13,No Defect_No Load,DECLINE,101.CA.DTL.01.1-030,DEC_TST,a8:ce:7d:2b,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-13,No Defect_No Load,DIVERT,101.CA.DTL.01.1-110,DIV_DAB,7c:92:dd:85,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-13,No Defect_No Load,METERING,101.CA.DTL.01.1-050,MET_TST,29:9c:fe:16,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# # load anand new file for clustering
# using Failure Induced column instead of Defects Introduced.

induced_flt = pd.read_excel('./Induced failure for cluster.xlsx', sheet_name='Induced Failures',header=0)
induced_flt.head(3)

,Entry Date,Defects Introduced,Failure Induced,Conveyor Type,Asset-Id,Sensor Location,Macid,Start Time,End Time,Time Difference,...,Roller_Wheels-Inner Trunk Roller,Roller_Wheels-Upper Trunk Roller,Roller_Wheels-RW Wheels,TensionUnit-RW-Bearing,TensionUnit-Tension Spring,TensionUnit-Tension Shaft / Unit,SlaveShaft-Bearing,SlaveShaft-Wheels,SlaveShaft- Shaft,SlaveShaft-Remarks2
0,2020-07-13,No Defect_No Load,No Defect_No Load,DECLINE,101.CA.DTL.01.1-030,DEC_TST,a8:ce:7d:2b,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-13,No Defect_No Load,No Defect_No Load,DIVERT,101.CA.DTL.01.1-110,DIV_DAB,7c:92:dd:85,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-13,No Defect_No Load,No Defect_No Load,METERING,101.CA.DTL.01.1-050,MET_TST,29:9c:fe:16,2020-07-13 15:00:00,2020-07-13 17:00:00,2 Hrs and 0 Min,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
old.columns

Index(['Entry Date', 'Defects Introduced', 'Conveyor Type', 'Asset-Id',
       'Sensor Location', 'Macid', 'Start Time', 'End Time', 'Time Difference',
       'Remarks', 'Test Condition', 'Drive Station', 'Belts', 'Rollers/Wheels',
       'Tension Station', 'Slave shafts', 'DriveShaft-Drive Motor',
       'DriveShaft-Drive Shaft', 'DriveShaft-Damper Bush',
       'DriveShaft-Cardan shaft ', 'DriveShaft-Bearing', 'Belt-Belt',
       'Roller_Wheels-Inner Trunk Roller', 'Roller_Wheels-Upper Trunk Roller',
       'Roller_Wheels-RW Wheels', 'TensionUnit-RW-Bearing',
       'TensionUnit-Tension Spring', 'TensionUnit-Tension Shaft / Unit',
       'SlaveShaft-Bearing', 'SlaveShaft-Wheels', 'SlaveShaft- Shaft',
       'SlaveShaft-Remarks2'],
      dtype='object')

In [5]:
induced_flt.columns

Index(['Entry Date', 'Defects Introduced', 'Failure Induced', 'Conveyor Type',
       'Asset-Id', 'Sensor Location', 'Macid', 'Start Time', 'End Time',
       'Time Difference', 'Remarks', 'Test Condition', 'Drive Station',
       'Belts', 'Rollers/Wheels', 'Tension Station', 'Slave shafts',
       'DriveShaft-Drive Motor', 'DriveShaft-Drive Shaft',
       'DriveShaft-Damper Bush', 'DriveShaft-Cardan shaft',
       'DriveShaft-Bearing', 'Belt-Belt', 'Roller_Wheels-Inner Trunk Roller',
       'Roller_Wheels-Upper Trunk Roller', 'Roller_Wheels-RW Wheels',
       'TensionUnit-RW-Bearing', 'TensionUnit-Tension Spring',
       'TensionUnit-Tension Shaft / Unit', 'SlaveShaft-Bearing',
       'SlaveShaft-Wheels', 'SlaveShaft- Shaft', 'SlaveShaft-Remarks2'],
      dtype='object')

In [6]:
# loop to load raw data and merge induced failures data from anand file and export as 
# Use only raw data from sensors that is listed in Anands file, else there would be error

for i in os.listdir('./Final_Raw_Data 26JulTo02Sep/'):
    Name = i[:8]
    df = pd.read_excel('./Final_Raw_Data 26JulTo02Sep/'+i)
    df_mac = pd.DataFrame()
    Mac_id = df.Mac_Id[0]
    Induced_filt = induced_flt[induced_flt.Macid == Mac_id]
    for i,row in df.iterrows():
        for j,flt in Induced_filt.iterrows():
            if row.Data_pull_time >= flt['Start Time'] and row.Data_pull_time <= flt['End Time']:
                New_row = row['Data_pull_time':'Axis']
                # print(type(flt[['Failure Induced','Sensor Location','Asset-Id']]))
                New_row = New_row.append(flt[['Failure Induced','Sensor Location','Asset-Id', 'Remarks', 'Test Condition', 'Drive Station', 'Belts', 'Rollers/Wheels',
       'Tension Station', 'Slave shafts', 'DriveShaft-Drive Motor',
       'DriveShaft-Drive Shaft', 'DriveShaft-Damper Bush',
       'DriveShaft-Cardan shaft', 'DriveShaft-Bearing', 'Belt-Belt',
       'Roller_Wheels-Inner Trunk Roller', 'Roller_Wheels-Upper Trunk Roller',
       'Roller_Wheels-RW Wheels', 'TensionUnit-RW-Bearing',
       'TensionUnit-Tension Spring', 'TensionUnit-Tension Shaft / Unit',
       'SlaveShaft-Bearing', 'SlaveShaft-Wheels', 'SlaveShaft- Shaft',
       'SlaveShaft-Remarks2']])
                New_row = New_row.append(row['Vsample1':'Vsample4096'])
                df_mac = df_mac.append(New_row,ignore_index=True)
    df_mac.to_excel('./Induced-Fault-Merged/'+Name+'_Fault_Induced.xlsx',index=False)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
df_mac.columns

Index([], dtype='object')

In [8]:
# Copying files to input folder "Induced-Fault-Merged2" from output "Induced-Fault-Merged"
# or copy manually and skip this

for i in os.listdir('./Induced-Fault-Merged'):
    Name = i[:8]
    df = pd.read_excel('./Induced-Fault-Merged/'+i)
    if df.shape[0] > 1:
        df[columns].to_excel('./Induced-Fault-Merged2/'+Name+'_Fault_Induced.xlsx',index=False)

KeyboardInterrupt: 

In [9]:
df_test = pd.read_excel('./Induced-Fault-Merged2/a8ce7d2b_Fault_Induced.xlsx',header=0)

### to process the faults induced data

In [8]:
for i in os.listdir('./Induced-Fault-Merged2'):
    Name = i[:8]
    Data = pd.read_excel('./Induced-Fault-Merged2/'+i)
    secs = 1.2792877225866917
    N = 4096
    fs_rate=round(N/secs)
    fs_rate
    Ts = 1.0/fs_rate # sampling interval in time
    t = np.arange(0, secs, Ts) # time vector as scipy arange field / numpy.ndarray
    freqs = scipy.fftpack.fftfreq(N, t[1]-t[0])
    freqs_side = freqs[range(N//2)] # one side frequency range
    ## high pass filter
    Data_Processed = pd.DataFrame()
    for i,row in Data.iterrows():
        data = row['Vsample1':'Vsample4096'] - row['Vsample1':'Vsample4096'].mean() # subracting mean from the samples
        #print(row["Data_pull_time":"Sensor_Loc"])
        f_signal = rfft(data) 
        cut_f_signal = f_signal.copy()
        data = irfft(cut_f_signal)
        rms = np.sqrt(np.mean(np.array(data)**2))
        kurt=stats.kurtosis(data)
        if rms>.02 and kurt<300 and freqs_side[-1]>1550: ## to keep the range at least 800 hz
            FFT = scipy.fft.fft(data)
            FFT_side = FFT[range(N//2)] # one side FFT range, because we need just the haf of the frequency data because of symmetry
            abs_fft_1=abs(FFT_side)
        
            payload={
                        "Data_pull_time" : row["Data_pull_time"],
                        "Mac_Id" : row["Mac_Id"],
                        "Axis" : row["Axis"],
                        "Defects Introduced" : row["Failure Induced"],
                        "Asset_Id" : row["Asset-Id"],
                        "Sensor_Loc" : row["Sensor Location"],
                
                        "Remarks" : row["Remarks"],
                        "Test Condition":row["Test Condition"],
                        "Drive Station" : row["Drive Station"],
                        "Belts": row["Belts"],
                        "Rollers/Wheels": row["Rollers/Wheels"],
                        "Tension Station": row["Tension Station"],
                        "Slave shafts": row["Slave shafts"],
                        "DriveShaft-Drive Motor": row["DriveShaft-Drive Motor"],
                        "DriveShaft-Drive Shaft": row["DriveShaft-Drive Shaft"],
                        "DriveShaft-Damper Bush'": row["DriveShaft-Damper Bush"],
                        "DriveShaft-Cardan shaft": row["DriveShaft-Cardan shaft"],
                        "DriveShaft-Bearing": row["DriveShaft-Bearing"],
                        "Belt-Belt": row["Belt-Belt"],
                        "Roller_Wheels-Inner Trunk Roller": row["Roller_Wheels-Inner Trunk Roller"],
                        "Roller_Wheels-Upper Trunk Roller": row["Roller_Wheels-Upper Trunk Roller"],
                        "Roller_Wheels-RW Wheels": row["Roller_Wheels-RW Wheels"],
                        "TensionUnit-RW-Bearing": row["TensionUnit-RW-Bearing"],
                        "TensionUnit-Tension Spring": row["TensionUnit-Tension Spring"],
                        "TensionUnit-Tension Shaft / Unit": row["TensionUnit-Tension Shaft / Unit"],
                        "SlaveShaft-Bearing": row["SlaveShaft-Bearing"],
                        "SlaveShaft-Wheels": row["SlaveShaft-Wheels"],
                        "SlaveShaft- Shaft": row["SlaveShaft- Shaft"],
                        "SlaveShaft-Remarks2": row["SlaveShaft-Remarks2"],
                
                        "peak-peak": (max(data)-min(data)),
                        "rms": rms,
                        "kurt":kurt,
                        "crest":np.max(np.abs(data))/np.sqrt(np.mean(np.square(data))),
                        "variance":np.var(data),
                        "mean":np.mean(data),
                        "skewness":stats.skew(data),
                        'freqAmpliMean':np.mean(abs_fft_1),
                        'freqAmpliMax':max(abs_fft_1),
                        'freqAmpliKurt':stats.kurtosis(abs_fft_1),
                        'freqAmpliskw':stats.skew(abs_fft_1),
                        'freqAmpliVar':np.var(abs_fft_1),
                        }
            lenFreq=len(freqs_side)
            lastFreq=freqs_side[-1]
            abs_fft_1=list(abs_fft_1)
            jj=0
            feqv=1
            while jj < lenFreq and feqv<=16: ## for 8 bins from 0 to 1600 Hz
                #print('Condition satisfied')
                kk=int(round(jj+lenFreq/lastFreq*100)) ## 100 is freqency step size for binnning
                #print(kk)
                fft_Range=abs_fft_1[jj:kk]
                freq_sideRange=freqs_side[jj:kk]
                jj=kk
                payload['max_{}'.format(feqv)]=max(fft_Range)
                payload['mean_{}'.format(feqv)]=np.mean(fft_Range)
                payload['var_{}'.format(feqv)]=np.var(fft_Range)
                feqv+=1
            Data_Processed = Data_Processed.append(payload, ignore_index=True)
    Data_Processed.to_excel('./Processed/'+Name+'_Processed.xlsx',index=False)
